RAG Pipeline-Data Ingestion to Vector DB

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
loader=DirectoryLoader("data/pdf_files", glob="*.pdf",loader_cls=PyMuPDFLoader)
data=loader.load()
print(data)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
dir_loader = DirectoryLoader("data/pdf_files", glob="*.pdf",loader_cls=PyMuPDFLoader)
dir_loader.load()